In [ ]:
!pip install langchain
!pip install openai
!pip install langchain_experimental
!pip install "langchain[docarray]"
!pip install chromadb

In [1]:
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import Chroma

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "please add your key here"


In [4]:
embedding_function = OpenAIEmbeddings()

loader = CSVLoader("/content/titanic_train.csv", encoding="windows-1252")
documents = loader.load()

db = Chroma.from_documents(documents, embedding_function)
retriever = db.as_retriever()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [5]:
# Create conversation chain
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4")
memory = ConversationBufferMemory(
memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        memory=memory
        )

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [8]:
query = "What is the ticket number for Braund, Mr. Owen Harris"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

'The ticket number for Braund, Mr. Owen Harris is A/5 21171.'

In [7]:
query = "What is the age of Heikkinen Miss Laina "
result = conversation_chain({"question": query})
answer = result["answer"]
answer

'Heikkinen, Miss. Laina was 26 years old.'